In [8]:
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm

In [9]:
TRAIN_DIR = '/notebooks/catsvsdogsdata/newdata/train'
TEST_DIR = '/notebooks/catsvsdogsdata/newdata/test'
IMG_SIZE = 50 #50x50 px
LEARNING_RATE = 1e-3

MODEL_NAME = 'facerec-{}-{}.model'.format(LEARNING_RATE, '2conv-light')

In [10]:
# defining labels for traning data
# using hot one technique
# [1,0] = cat in img & [0,1] = dog in img
def label_img(img):
    word_label = img.split('.')[-3]
    if word_label == 'cat': return [1,0]
    elif word_label == 'dog': return [0,1]

In [11]:
# convert feature to 2d greyscale data
# creates the traning data with labels -> cat = [[[1,0],[greyscale img data]]], dog = [[[0,1],[greyscale img data]]]
# whole traning data example (with first cat and second dog): [[[1,0],[greyscale img data]], [[0,1],[greyscale img data]]]
# images gets resized to 50x50px
# the traning data gets shuffled to make sure that they are all random and we dont have any sitiuations where all the
# cats come first and after the dogs
# the whole data gets saved out to 'traning_data.npy' file
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE))
        training_data.append([np.array(img), np.array(label)])
        
    shuffle(training_data)
    np.save('traning_data.npy', training_data)
    return training_data

In [12]:
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR,img)
        img_num = img.split('.')[0]
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE))
        testing_data.append([np.array(img), img_num])
        
    np.save('test_data.npy', testing_data)
    return testing_data

In [13]:
train_data = create_train_data()

# if the data already is processed simply load it
# train_data = np.load('train_data.npy')

100%|██████████| 25000/25000 [03:37<00:00, 114.76it/s] 


In [15]:
# Setting up the CNN with tflearn
# using two conv. layers with max pooling
# a droprate of 80%
# activation function is set to relu
# final activation function is softmax since we want multi-classification
# optimize algorithm is adam and loss algorithm is categorical crossentropy
# both the optimizer and loss functions are used for back-prop
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LEARNING_RATE, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')

In [16]:
# Check if we already have trained on a model, if so load the weights and continue
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('Model loaded!')

In [18]:
# Seperating data to traning data and test data
train = train_data[:-500]
test = train_data[-500:]

In [22]:
# traning data
# feature sets
X = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
# labels
Y = [i[1] for i in train]

# test data (for accuracy)
# feature sets
test_x = np.array([i[0] for i in test]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
# labels
test_y = [i[1] for i in test]

In [23]:
model.fit({'input': X}, {'targets': Y}, n_epoch=5, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

Training Step: 1914  | total loss: 11.61867 | time: 415.709s
| Adam | epoch: 005 | loss: 11.61867 - acc: 0.4954 -- iter: 24448/24500
Training Step: 1915  | total loss: 11.50016 | time: 418.319s
| Adam | epoch: 005 | loss: 11.50016 - acc: 0.5006 | val_loss: 11.65108 - val_acc: 0.4940 -- iter: 24500/24500
--


In [25]:
# command for running tensorboard
# tensorboard --logdir='log'

In [ ]:
model.save(MODEL_NAME)